In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = 'Data/'
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"

In [3]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if re.match(r"ukwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_art.csv.gz", file):
        UKWIKI_ART_FNMS.append(file)  

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [17]:
fn = PATH_TO_DATA_UK+UKWIKI_ART_FNMS[0]
print(fn)
fn_new = unpack(fn)
df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
print(df_articles.shape)
df_red_links = df_articles[df_articles['is_red_link'] == True]
print(df_red_links.shape)

Data/ukwiki/ukwiki-20180620-pages-meta-current01-p1p5501930.xml_art.csv.gz
(5532654, 6)
(4681130, 6)


In [6]:
df_title_count = None
for fn in UKWIKI_ART_FNMS:
    fn = PATH_TO_DATA_UK+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")

    df_red_links = df_articles[df_articles['is_red_link'] == True]
    df_title_count_tmp = pd.DataFrame(df_red_links.groupby('link_val').link_val.count())
    df_title_count_tmp.columns = ['link']
    df_title_count_tmp = df_title_count_tmp.reset_index(col_level=1)
    df_title_count_tmp.columns = ['link_title', 'in_count']
    
    if df_title_count is not None:
        df_title_count = df_title_count.append(df_title_count_tmp)        
        #print("append")
    else:
        df_title_count = df_title_count_tmp
        #print("assign")    
    df_title_count = df_title_count.groupby('link_title').in_count.sum().copy()
    df_title_count.columns = ['link']
    df_title_count = df_title_count.reset_index()
    df_title_count.columns = ['link_title', 'in_count']        
    print("df_title_count size: {}".format(df_title_count.shape))
    os.remove(fn_new)

Data/ukwiki/ukwiki-20180620-pages-meta-current01-p1p5501930.xml_art.csv.gz
df_title_count size: (839717, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current02-p5501931p11003859.xml_art.csv.gz
df_title_count size: (1387130, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current03-p11003860p16505788.xml_art.csv.gz
df_title_count size: (1959796, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current04-p16505789p22007717.xml_art.csv.gz
df_title_count size: (2219249, 2)


In [7]:
df_title_count = df_title_count.sort_values(['in_count'], ascending=[0])

In [8]:
df_title_count.to_csv(PATH_TO_DATA_UK+'ukwiki-20180620-red_name_count.csv', index = False, encoding='UTF-8', quotechar="\"")

In [9]:
df_title_count

,link_title,in_count
1105325,Лейцин,216398
1627444,Серин,182784
687315,Глутамінова кислота,157676
375181,Аланін,150468
688288,Гліцин,142450
1486605,Пролін,136950
545260,Валін,131401
1147028,Лізин,128320
419655,Аргінін,123965
1766627,Треонін,117366


In [10]:
df_title_count.describe()

,in_count
count,2.219249e+06
mean,8.601627e+00
std,3.871414e+02
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,3.000000e+00
max,2.163980e+05


In [11]:
df_count_by_count = pd.DataFrame(df_title_count.groupby('in_count').in_count.count())

In [12]:
df_count_by_count.columns = ['link']
df_count_by_count = df_count_by_count.reset_index()
df_count_by_count.columns = ['count', 'in_count']  

In [13]:
df_count_by_count = df_count_by_count.sort_values(['count'], ascending=[1])

In [14]:
df_count_by_count.head()

,count,in_count
0,1,1275888
1,2,315278
2,3,147089
3,4,89494
4,5,62652


In [15]:
df_count_by_count.describe()

,count,in_count
count,2058.000000,2.058000e+03
mean,3098.760447,1.078352e+03
std,12286.086081,2.928908e+04
min,1.000000,1.000000e+00
25%,515.250000,1.000000e+00
50%,1092.500000,2.000000e+00
75%,1957.000000,7.000000e+00
max,216398.000000,1.275888e+06


In [16]:
df_count_by_count.to_csv(PATH_TO_DATA_UK+'ukwiki-20180620-red_count_by_count.csv', index = False, encoding='UTF-8', quotechar="\"")